In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Duryan\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy


# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# NLTK stopwords
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(["from", "subject", "re", "edu", "use"])

In [4]:
df_indeed = pd.read_csv("data/indeed_data_scientist.csv")
df_linkedin = pd.read_csv("data/linkedin_data_scientist.csv")
df = pd.concat([df_indeed, df_linkedin], axis=0)
df.drop_duplicates(subset=["title", "company", "text"], keep="first", inplace=True)
df.dropna(subset=["text"], inplace=True)
df.reset_index(inplace=True, drop=True)

In [5]:
data = df.text.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]


In [6]:
# print(df["text"].loc[0])

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['bay', 'street', 'canada', 'toronto', 'toronto', 'ontario', 'at', 'capital', 'one', 'we', 're', 'building', 'leading', 'information', 'based', 'technology', 'company', 'still', 'founder', 'led', 'by', 'chairman', 'and', 'chief', 'executive', 'officer', 'richard', 'fairbank', 'capital', 'one', 'is', 'on', 'mission', 'to', 'help', 'our', 'customers', 'succeed', 'by', 'bringing', 'ingenuity', 'simplicity', 'and', 'humanity', 'to', 'banking', 'we', 'measure', 'our', 'efforts', 'by', 'the', 'success', 'our', 'customers', 'enjoy', 'and', 'the', 'advocacy', 'they', 'exhibit', 'we', 'are', 'succeeding', 'because', 'they', 'are', 'succeeding', 'guided', 'by', 'our', 'shared', 'values', 'we', 'thrive', 'in', 'an', 'environment', 'where', 'collaboration', 'and', 'openness', 'are', 'valued', 'we', 'believe', 'that', 'innovation', 'is', 'powered', 'by', 'perspective', 'and', 'that', 'teamwork', 'and', 'respect', 'for', 'each', 'other', 'lead', 'to', 'superior', 'results', 'we', 'elevate', 'each',

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['bay_street', 'canada', 'toronto', 'toronto', 'ontario', 'at', 'capital_one', 'we', 're', 'building', 'leading', 'information', 'based', 'technology', 'company', 'still', 'founder', 'led', 'by', 'chairman', 'and', 'chief', 'executive', 'officer', 'richard', 'fairbank', 'capital_one', 'is', 'on', 'mission', 'to', 'help', 'our', 'customers', 'succeed', 'by', 'bringing', 'ingenuity', 'simplicity', 'and', 'humanity', 'to', 'banking', 'we', 'measure', 'our', 'efforts', 'by', 'the', 'success', 'our', 'customers', 'enjoy', 'and', 'the', 'advocacy', 'they', 'exhibit', 'we', 'are', 'succeeding', 'because', 'they', 'are', 'succeeding', 'guided', 'by', 'our', 'shared', 'values', 'we', 'thrive', 'in', 'an', 'environment', 'where', 'collaboration', 'and', 'openness', 'are', 'valued', 'we', 'believe', 'that', 'innovation', 'is', 'powered', 'by', 'perspective', 'and', 'that', 'teamwork', 'and', 'respect', 'for', 'each', 'other', 'lead', 'to', 'superior', 'results', 'we', 'elevate', 'each', 'other', 

In [9]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [10]:
# # Remove Stop Words
# data_words_nostops = remove_stopwords(data_words)

# # Form Bigrams
# data_words_bigrams = make_bigrams(data_words_nostops)

# # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# # python3 -m spacy download en
# # nlp = spacy.load('en_trf_xlnetbasecased_lg', disable=['parser', 'ner'])
# nlp = spacy.load('en_trf_xlnetbasecased_lg')


# # Do lemmatization keeping only noun, adj, vb, adv
# data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# # print(data_lemmatized)

In [15]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized)

[['building', 'lead', 'information', 'base', 'technology', 'company', 'still', 'founder', 'lead', 'chairman', 'chief', 'mission', 'help', 'customer', 'succeed', 'bring', 'ingenuity', 'simplicity', 'humanity', 'banking', 'measure', 'effort', 'success', 'customer', 'enjoy', 'advocacy', 'exhibit', 'succeed', 'succeed', 'guide', 'shared', 'value', 'thrive', 'environment', 'collaboration', 'openness', 'value', 'believe', 'innovation', 'power', 'perspective', 'teamwork', 'respect', 'lead', 'superior', 'result', 'elevate', 'obsess', 'right_thing', 'associate', 'serve', 'humility', 'deep', 'respect', 'responsibility', 'help', 'customer', 'achieve', 'goal', 'realize', 'dream', 'together', 'quest', 'change', 'bank', 'good', 'datum', 'scientist', 'help', 'million', 'canadian', 'year', 'provide', 'access', 'credit', 'journey', 'put', 'customer', 'first', 'keep', 'provide', 'innovative', 'smart', 'solution', 'make', 'banking', 'effortless', 'work', 'capital', 'think', 'datum', 'drive', 'tech', 'com

In [16]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 3), (10, 2), (11, 1), (12, 1), (13, 2), (14, 2), (15, 3), (16, 2), (17, 1), (18, 1), (19, 1), (20, 2), (21, 1), (22, 1), (23, 3), (24, 2), (25, 3), (26, 1), (27, 1), (28, 6), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 3), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 2), (51, 1), (52, 1), (53, 1), (54, 1), (55, 6), (56, 1), (57, 1), (58, 14), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 1), (79, 2), (80, 1), (81, 1), (82, 3), (83, 1), (84, 1), (85, 1), (86, 1), (87, 4), (88, 1), (89, 1), (90, 2), (91, 1), (92, 2), (93, 1), (94, 1), (95, 1), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 2), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 4

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.027*"shopify" + 0.021*"water" + 0.018*"cell" + 0.016*"merchant" + '
  '0.014*"environmental" + 0.014*"product" + 0.012*"technical" + '
  '0.012*"laboratory" + 0.011*"sample" + 0.011*"application"'),
 (1,
  '0.019*"team" + 0.016*"product" + 0.015*"work" + 0.014*"datum" + '
  '0.010*"drive" + 0.009*"help" + 0.009*"company" + 0.009*"experience" + '
  '0.009*"marketing" + 0.008*"customer"'),
 (2,
  '0.019*"sale" + 0.019*"experience" + 0.016*"skill" + 0.014*"system" + '
  '0.011*"project" + 0.010*"high" + 0.009*"market" + 0.008*"time" + '
  '0.008*"pair" + 0.008*"industry"'),
 (3,
  '0.028*"pour" + 0.021*"vous" + 0.021*"analytic" + 0.020*"nous" + '
  '0.018*"trading" + 0.013*"insight" + 0.011*"poste" + 0.011*"advanced" + '
  '0.010*"aux" + 0.009*"que"'),
 (4,
  '0.000*"datum" + 0.000*"experience" + 0.000*"work" + 0.000*"research" + '
  '0.000*"year" + 0.000*"scientist" + 0.000*"learning" + 0.000*"science" + '
  '0.000*"skill" + 0.000*"help"'),
 (5,
  '0.025*"datum" + 0.022*"manage

In [24]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds='mmds')
vis

E:\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9      0.302454  0.036371       1        1  39.177582
1      0.248307  0.121801       2        1  19.443388
5      0.321985 -0.052427       3        1  12.792148
10     0.347551  0.159438       4        1  12.241027
13     0.199557 -0.297045       5        1   4.607140
2      0.260294 -0.199296       6        1   2.882837
17     0.159899  0.301008       7        1   2.156346
6      0.022821 -0.408886       8        1   2.125429
16     0.084097  0.239126       9        1   2.094488
0     -0.149575  0.272077      10        1   0.930559
3     -0.014584 -0.175773      11        1   0.808349
12    -0.319771  0.193076      12        1   0.439315
15    -0.203770 -0.114416      13        1   0.220539
11    -0.197573 -0.007743      14        1   0.036427
7     -0.184878 -0.021142      15        1   0.027533
18    -0.175804 -0.008673      16        1   0.005003
4     -0.175253 -0.009374      17        1   0.003041
14    -0.175253 -0.009374      18        1   0.003039
8     -0.175253 -0.009374      19        1   0.002943
19    -0.175253 -0.009374      20        1   0.002878, topic_info=     Category         Freq        Term        Total  loglift  logprob
151   Default  1005.000000       model  1005.000000  30.0000  30.0000
58    Default  4978.000000       datum  4978.000000  29.0000  29.0000
538   Default   844.000000  management   844.000000  28.0000  28.0000
293   Default   352.000000   candidate   352.000000  27.0000  27.0000
87    Default  3204.000000  experience  3204.000000  26.0000  26.0000
...       ...          ...         ...          ...      ...      ...
2249  Topic20     0.000719        list    27.723057  -0.1035  -8.8208
2248  Topic20     0.000719    intended     1.651060   2.7174  -8.8208
2247  Topic20     0.000719      intend     8.416457   1.0886  -8.8208
2234  Topic20     0.000719      choice    17.606506   0.3505  -8.8208
2224  Topic20     0.000719      mutual     5.674464   1.4828  -8.8208

[5189 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
777       1  0.434164  ability
777       2  0.122420  ability
777       3  0.145196  ability
777       4  0.044128  ability
777       5  0.135232  ability
...     ...       ...      ...
265       5  0.037431     year
265       6  0.029410     year
776       5  0.685323      yet
776      13  0.228441      yet
3409      5  0.973245    youth

[2053 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 2, 6, 11, 14, 3, 18, 7, 17, 1, 4, 13, 16, 12, 8, 19, 5, 15, 9, 20])